[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wY9N4XrpOf65EI01LDFdSEEkOFwJ18JT?usp=sharing)


In [1]:
import nbformat

# Load the current notebook
notebook_path = "/content/Untitled2.ipynb"  # Change this if needed
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Clean metadata.widgets
if "widgets" in nb["metadata"]:
    if "state" not in nb["metadata"]["widgets"]:
        print("Removing invalid 'metadata.widgets' without 'state'")
        del nb["metadata"]["widgets"]

# Save the cleaned notebook
with open("/content/cleaned_notebook.ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("Cleaned notebook saved as cleaned_notebook.ipynb")


FileNotFoundError: [Errno 2] No such file or directory: '/content/Untitled2.ipynb'

In [ ]:
!pip install transformers datasets sentencepiece evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=63f0af55c45e83b4c31d335700a3b058365bb432bf0a3413f4ee375e4ded5ec4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's depend

In [ ]:
from sentence_transformers import SentenceTransformer, util
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load NER model using XLM-RoBERTa
model_name = "xlm-roberta-large-finetuned-conll03-english"
tokenizer_ner = AutoTokenizer.from_pretrained(model_name)
model_ner = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, aggregation_strategy="simple")

# Create NER pipeline
#ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def extract_named_entities(text):
    ner_results = ner_pipeline(text)
    entities = set([ent['word'] for ent in ner_results if ent['entity_group'] in ["PER", "ORG", "LOC", "EVENT"]])
    return entities
# Load sentence transformer (multilingual, supports Telugu)
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")  # or use "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

def smart_insert_entity(summary, entity, original_text):
    # Split summary and original into sentences
    summary_sents = sent_tokenize(summary)
    original_sents = sent_tokenize(original_text)

    # Compute embeddings
    summary_embeddings = embedder.encode(summary_sents, convert_to_tensor=True)
    original_embeddings = embedder.encode(original_sents, convert_to_tensor=True)

    # Find sentence in original text where entity is found
    for i, sent in enumerate(original_sents):
        if entity in sent:
            query_embedding = original_embeddings[i]
            break
    else:
        return summary  # If entity not in original, return original summary

    # Find best matching summary sentence
    cos_scores = util.cos_sim(query_embedding, summary_embeddings)[0]
    best_idx = int(cos_scores.argmax())

    # Insert entity into the most relevant sentence if it's not already there
    if entity not in summary_sents[best_idx]:
        summary_sents[best_idx] = entity + ": " + summary_sents[best_idx]

    return " ".join(summary_sents)

def ner_postprocess_smart(original_text, generated_summary):
    original_entities = extract_named_entities(original_text)
    summary_entities = extract_named_entities(generated_summary)
    missing_entities = original_entities - summary_entities

    enhanced_summary = generated_summary
    for entity in missing_entities:
        enhanced_summary = smart_insert_entity(enhanced_summary, entity, original_text)

    return enhanced_summary

original_text ="""ఆంధ్రప్రదేశ్ ఉప ముఖ్యమంత్రి పవన్ కల్యాణ్ కూతురు ఆద్య ఆటోలో ప్రయాణిస్తున్న వీడియో ప్రస్తుతం సోషల్ మీడియాలో వైరల్ గా మారింది. పవన్ కల్యాణ్ మాజీ భార్య, సినీ నటి రేణు దేశాయ్ ఈ వీడియోను తన ఇన్ స్టా అకౌంట్ లో పోస్ట్ చేశారు. కాశీ (వారణాసి) లో ఆద్యతో ఆటో రైడ్ అంటూ రేణు షేర్ చేశారీ వీడియో. ఇక ఈ వీడియో చూసిన పవన్ అభిమానులు కామెంట్ల రూపంలో తమ అభిమానాన్ని కురిపిస్తున్నారు. తండ్రి ఓ రాష్ట్రానికి ఉప ముఖ్యమంత్రి అయినా కూతురు మాత్రం ఎంత సింపుల్ గా నడుచుకుంటుందో చూడండి అంటూ అభిమానులు మురిసిపోతున్నారు. ఆద్య నిరాడంబరతను మెచ్చుకుంటూ కామెంట్లు పెడుతున్నారు. ఆద్య, అకిరాలను ఆడంబరాలు, విలాసాలకు అతీతంగా తల్లి రేణు దేశాయ్ సాధారణంగా పెంచడంపై ప్రశంసలు కురిపిస్తున్నారు."""
generated_summary ="""ఆంధ్రప్రదేశ్ ఉప ముఖ్యమంత్రి పవన్ కల్యాణ్ కూతురు ఆద్య ఆటోలో ప్రయాణిస్తున్న వీడియో సోషల్ మీడియాలో వైరల్ గా మారింది. దీనిపై ఆయన మాజీ భార్య రేణు దేశాయ్ స్పందించారు."""
summ = ner_postprocess_smart(original_text, generated_summary)
print(summ)
#df['Entity-based Abstractive Summary'] = df.apply(lambda row: ner_postprocess_smart(" ".join(row['Text']), row['Entity']) if isinstance(row['Text'], list) else generate_abstractive_summary(row['Text'], row['Entity']), axis=1)

#df_result = df[['Text', 'Entity', 'Reference summary','Entity-based Summary','Entity-based Abstractive Summary']]
# Save results
#df_result = df[['Text', 'Entity', 'Named Entities BIO', 'Entity-based Summary', 'Entity-based Abstractive Summary']]
#df_result.to_csv('telugu_entity_abstractive.csv', index=False)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


అకిరా: రేణు దేశ: కాశీ: వారణాసి: రేణు: ఆంధ్రప్రదేశ్ ఉప ముఖ్యమంత్రి పవన్ కల్యాణ్ కూతురు ఆద్య ఆటోలో ప్రయాణిస్తున్న వీడియో సోషల్ మీడియాలో వైరల్ గా మారింది. దీనిపై ఆయన మాజీ భార్య రేణు దేశాయ్ స్పందించారు.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load CSV dataset
dataset = load_dataset("csv", data_files={"train": "dataset2.csv"}, split="train")

# Split into train and validation
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset['train']
val_data = dataset['test']

# Load mT5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess(batch):
    inputs = tokenizer(batch["Text"], max_length=max_input_length, truncation=True, padding="max_length")
    targets = tokenizer(batch["Reference Summary"], max_length=max_target_length, truncation=True, padding="max_length")

    inputs["labels"] = targets["input_ids"]
    return inputs

train_dataset = train_data.map(preprocess, batched=True, remove_columns=train_data.column_names)
val_dataset = val_data.map(preprocess, batched=True, remove_columns=val_data.column_names)


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
from transformers import MT5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import os
# ✅ Disable Weights & Biases logging to avoid API key prompt
os.environ["WANDB_DISABLED"] = "true"
model = MT5ForConditionalGeneration.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5_telugu_summarizer",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Use if GPU available
    report_to=None,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


<ipython-input-6-bbb18fc7af2a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import os
# ✅ Disable Weights & Biases logging to avoid API key prompt
os.environ["WANDB_DISABLED"] = "true"

# Load pretrained mT5 model
model_name = "csebuetnlp/mT5_multilingual_XLSum"  # Or use other sizes like 'mT5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


# Load CSV dataset
dataset = load_dataset("csv", data_files={"train": "dataset2.csv"}, split="train")

# Tokenizing the dataset
def preprocess_data(examples):
    inputs = tokenizer(examples['Text'], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples['Reference Summary'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

# Apply the preprocessing to your dataset
train_dataset = [preprocess_data(item) for item in dataset]

# Fine-tuning the model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset  # Use a separate validation dataset for real use
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a mode

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.nn as nn
import torch
import pandas as pd
df = pd.read_csv("dataset2.csv")

# Load pretrained mT5 model
model_name = "csebuetnlp/mT5_multilingual_XLSum"  # Or use other sizes like 'mT5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# Custom Dataset
class SummaryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_len=512, max_output_len=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = f"summarize: {row['Entity-based Summary']}"
        target_text = row['Reference Summary']

        input_enc = self.tokenizer(
            input_text, padding='max_length', truncation=True,
            max_length=self.max_input_len, return_tensors="pt"
        )
        target_enc = self.tokenizer(
            target_text, padding='max_length', truncation=True,
            max_length=self.max_output_len, return_tensors="pt"
        )

        labels = target_enc.input_ids.squeeze()
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss

        return {
            'input_ids': input_enc.input_ids.squeeze(),
            'attention_mask': input_enc.attention_mask.squeeze(),
            'labels': labels
        }

# Prepare dataset
train_dataset = SummaryDataset(df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optimizer & Loss
optimizer = AdamW(model.parameters(), lr=3e-5)
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# Training loop
epochs = 3
model.train()

for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/FineTuned_mT5_Telugu")
tokenizer.save_pretrained("/content/drive/MyDrive/FineTuned_mT5_Telugu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a mode

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/FineTuned_mT5_Telugu")
